In [15]:
import pandas as pd
import numpy as np
import pickle
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, entropy
from scipy.fft import fft

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
person = "Miguel" #person's observation

#Creation of new features

In [18]:
#----------------------------------------------------------------------------------
#--------------------------------
# Mobile sensor data acquisition and processing
#----------------------------------------------------------------------------------
#--------------------------------

# Time-domain additional features
def time_features(sig):
    median_ = np.median(sig)
    min_   = np.min(sig)
    max_ =  np.max(sig)
    range_ = max_ - min_
    return [median_, min_, max_, range_]

# Frequency-domain features
def frequency_features(sig, sampling_rate=50):
    N = len(sig)
    freqs = np.fft.fftfreq(N, d=1.0 / sampling_rate)
    fft_vals = np.abs(fft(sig))
    fft_vals = fft_vals[:N // 2]
    freqs = freqs[:N // 2]

    if np.sum(fft_vals) == 0:
        spectral_entropy = 0
        mean_freq = 0
    else:
        norm_fft = fft_vals / np.sum(fft_vals)
        spectral_entropy = entropy(norm_fft)
        mean_freq = np.sum(freqs * norm_fft)

    peak_freq = freqs[np.argmax(fft_vals)]
    fft_energy = np.sum(fft_vals ** 2)

    return [peak_freq, spectral_entropy, mean_freq, fft_energy]

#loading the obj
with open("/content/drive/MyDrive/AI_Modeling/Project1/"+person+".obj", "rb") as f:
    experiment_data = pickle.load(f)

features = []

for activity, label_id, window in experiment_data:
    feat = [label_id]
    rms = 0
    for i in range(3):
        sig = window[:, i]
        feat.append(np.mean(sig))
        feat.append(np.std(sig))
        feat.append(kurtosis(sig))
        feat.append(skew(sig))
        rms += np.sum(sig ** 2)
    #RMS
    feat.append(np.sqrt(rms))

    # median, min, max, etc
    for i in range(3):
        sig = window[:, i]
        feat.extend(time_features(sig))

    # Frequency
    for i in range(3):
        sig = window[:, i]
        feat.extend(frequency_features(sig))

    features.append(feat)


processed_data = np.array(features)
x = processed_data[:, 1:]  # features only
y = processed_data[:, 0]   # labels

stat_cols = []
for ax in 'xyz':
    for stat in ['mean', 'std', 'kurtosis', 'skew']:
        stat_cols.append(f"{stat}_{ax}")

rms_col = ['rms']

time_cols = []
for ax in 'xyz':
    for stat in ['median', 'min', 'max', 'range']:
        time_cols.append(f"{stat}_{ax}")

freq_cols = []
for ax in 'xyz':
    for stat in ['peak_freq', 'spectral_entropy', 'mean_freq', 'fft_energy']:
        freq_cols.append(f"{stat}_{ax}")

all_columns = ['label']
all_columns.extend(stat_cols)
all_columns.extend(rms_col)
all_columns.extend(time_cols)
all_columns.extend(freq_cols)
df = pd.DataFrame(processed_data, columns=all_columns)




df.head()

#----------------------------------------------------------------------------------
#--------------------------------
# End of file
#----------------------------------------------------------------------------------
#--------------------------------


,label,mean_x,std_x,kurtosis_x,skew_x,mean_y,std_y,kurtosis_y,skew_y,mean_z,...,mean_freq_x,fft_energy_x,peak_freq_y,spectral_entropy_y,mean_freq_y,fft_energy_y,peak_freq_z,spectral_entropy_z,mean_freq_z,fft_energy_z
0,5.0,2.557499,1.293377,-0.999987,-0.487717,-16.010095,7.610073,-0.953773,-0.705175,2.128833,...,2.706573,4608.288143,0.0,1.398635,2.346362,178240.957660,0.0,2.170947,6.117145,4475.303540
1,5.0,-6.056320,8.432990,-0.704669,-0.730033,-8.858947,9.819147,-0.374209,-0.671862,0.783023,...,3.306134,45139.897151,0.0,2.059461,5.231609,78848.424072,2.0,1.438719,3.731999,14646.615494
2,5.0,-2.932846,5.127624,-1.172847,0.260777,-9.667458,6.382217,-1.365210,0.140228,-1.259893,...,2.354842,13591.424483,0.0,1.278502,1.984829,71130.510787,2.0,1.848041,4.234959,3239.070683
3,5.0,1.094053,0.976997,-1.265044,-0.476306,-11.491232,3.735004,-1.330727,-0.441262,1.641208,...,2.129340,1046.324140,0.0,1.107918,1.583865,86883.112760,0.0,1.766198,3.961246,2640.503680
4,5.0,2.283141,1.668680,-0.582491,0.038275,-15.094841,4.849610,-1.573934,0.036933,0.825570,...,2.872116,4126.720178,0.0,0.928275,1.140342,149754.495580,2.0,1.820273,5.129908,7840.602154


In [19]:
print(f"Number of features: {len(df.columns)-1}")

Number of features: 37


In [20]:
df.columns

Index(['label', 'mean_x', 'std_x', 'kurtosis_x', 'skew_x', 'mean_y', 'std_y',
       'kurtosis_y', 'skew_y', 'mean_z', 'std_z', 'kurtosis_z', 'skew_z',
       'rms', 'median_x', 'min_x', 'max_x', 'range_x', 'median_y', 'min_y',
       'max_y', 'range_y', 'median_z', 'min_z', 'max_z', 'range_z',
       'peak_freq_x', 'spectral_entropy_x', 'mean_freq_x', 'fft_energy_x',
       'peak_freq_y', 'spectral_entropy_y', 'mean_freq_y', 'fft_energy_y',
       'peak_freq_z', 'spectral_entropy_z', 'mean_freq_z', 'fft_energy_z'],
      dtype='object')

In [21]:
df.to_csv("/content/drive/MyDrive/AI_Modeling/Project1/"+person+"_all_features.csv", index=False)